Monthly Electricity Data from U.S. Energy Information Administration (EIA), and Hourly Weather Data from the U.S. National Oceanic & Atmospheric Administration.

In [225]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
%matplotlib inline

from scipy.integrate import simps

2006 temperature data was erroneous (always less than 10 degrees), so it was excluded.
Other and Transportation electricity data was relatively small and not present for all years, so it was excluded.

In [35]:
CDD_BASE_TEMP = 18

In [169]:
# temperature_train_df = pd.read_csv('DC_temperature_data_2001-2018_july.csv')
# temperature_test_df = pd.read_csv('DC_temperature_data_2001-2018_august.csv')
# electricity_train_df = pd.read_csv('Retail_sales_of_electricity_july.csv')
# electricity_test_df = pd.read_csv('Retail_sales_of_electricity_august.csv')

In [170]:
# temperature_train_df['cdd_rel_temp'] = temperature_train_df.air_temp_c.apply(lambda x: max((x-CDD_BASE_TEMP)/24,0))
# temperature_test_df['cdd_rel_temp'] = temperature_test_df.air_temp_c.apply(lambda x: max((x-CDD_BASE_TEMP)/24,0))

In [171]:
# cdd_annual_train = temperature_train_df[['year', 'cdd_rel_temp']].groupby(by='year').sum()
# cdd_annual_test = temperature_test_df[['year', 'cdd_rel_temp']].groupby(by='year').sum()

In [172]:
# temperature_train_df[['hour', 'air_temp_c']].groupby(by='hour').mean().plot()
# plt.grid()
# plt.xticks(list(temperature_train_df.hour.unique()));

In [173]:
# cdd_annual_train['avg_temp'] = temperature_train_df[['year', 'air_temp_c']].groupby(by='year').mean()
# cdd_annual_test['avg_temp'] = temperature_test_df[['year', 'air_temp_c']].groupby(by='year').mean()

In [174]:
# cdd_annual_train['avg_daily_max_temp'] = temperature_train_df[['year',
#                                                                'day',
#                                                                'air_temp_c']].groupby(by=['year',
#                                                                                           'day']).max().groupby(by='year').mean()
# cdd_annual_test['avg_daily_max_temp'] = temperature_test_df[['year',
#                                                              'day',
#                                                              'air_temp_c']].groupby(by=['year',
#                                                                                         'day']).max().groupby(by='year').mean()

In [175]:
# # time data is offset, appears to be by ~6 hours
# cdd_annual_train['avg_noon_temp'] = temperature_train_df.loc[(temperature_train_df.hour == 18),
#                                                              ['year', 'air_temp_c']].groupby(by='year').mean()
# cdd_annual_test['avg_noon_temp'] = temperature_test_df.loc[(temperature_test_df.hour == 18),
#                                                            ['year', 'air_temp_c']].groupby(by='year').mean()

In [176]:
# cdd_annual_train.reset_index(inplace=True)
# cdd_annual_test.reset_index(inplace=True)

In [177]:
# train_df = electricity_train_df.merge(cdd_annual_train)
# test_df = electricity_test_df.merge(cdd_annual_test)

In [178]:
# train_df = train_df.loc[train_df.year > 2008]
# test_df = test_df.loc[test_df.year > 2008]

In [233]:
data_df = pd.read_csv('building_electricity_consumption.csv')
cdd_df = pd.read_csv('building_electricity_consumption_for_integration.csv')

In [301]:
cdd_df = pd.read_csv('building_electricity_consumption_for_integration.csv')

In [302]:
cdd_df['date'] = cdd_df.year.astype('str')+cdd_df.month.astype('str').str.zfill(2)+cdd_df.day.astype('str').str.zfill(2)
cdd_df.date = cdd_df.date.astype('int')
cdd_df.drop(columns=['kW'], inplace=True)
first_day = cdd_df[cdd_df.hour == 1].reset_index(drop=True).iloc[1:]
first_day.reset_index(inplace=True, drop=True)
first_day.hour = 25

In [303]:
first_day.index

RangeIndex(start=0, stop=354, step=1)

In [304]:
# transition between months and years isn't working right
prev_month = {1: 12, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11}
prev_month_length = {2: 31, 3: 28, 4: 31, 5: 30, 6: 31, 7:30, 8: 31, 9: 31, 10:30, 11:31, 12: 30, 1:31}
for i in first_day.index:
    if first_day.date.iloc[i] == 20170101:
        first_day.day.iloc[i] = 31
        first_day.month.iloc[i] = 12
        first_day.year.iloc[i] = 2016
    else:
        if first_day.day.iloc[i] == 1:
            first_day.day.iloc[i] = prev_month_length[first_day.month.iloc[i]]
            first_day.month.iloc[i] = prev_month[first_day.month.iloc[i]]
        else:
            first_day.day.iloc[i] -= 1

first_day.date = first_day.year.astype('str')+first_day.month.astype('str').str.zfill(2)+first_day.day.astype('str').str.zfill(2)

In [305]:
cdd_df = pd.concat([cdd_df, first_day])
cdd_df.reset_index(inplace=True, drop=True)
cdd_df.sort_values(by=['date', 'hour'], inplace=True)

In [306]:
first_day.head()

,year,month,day,hour,temp,date
0,2016,8,25,25,80.1,20160825
1,2016,8,26,25,82.0,20160826
2,2016,8,27,25,78.1,20160827
3,2016,8,28,25,77.0,20160828
4,2016,8,29,25,81.0,20160829


In [307]:
cdd_df.head(25)

,year,month,day,hour,temp,date
0,2016,8,25,1,73.9,20160825
1,2016,8,25,2,72.0,20160825
2,2016,8,25,3,72.0,20160825
3,2016,8,25,4,72.0,20160825
4,2016,8,25,5,72.0,20160825
5,2016,8,25,6,71.1,20160825
6,2016,8,25,7,71.1,20160825
7,2016,8,25,8,73.0,20160825
8,2016,8,25,9,75.9,20160825
9,2016,8,25,10,80.1,20160825


In [308]:
cdd_df['month_day'] = cdd_df.month.astype('str')+'-'+cdd_df.day.astype('str')

In [309]:
cdd_df['cdd'] = cdd_df.temp.apply(lambda x: max(round((x-CDD_BASE_TEMP)/24,4),0))

In [310]:
cdd_groupby = cdd_df[['month_day', 'cdd']].groupby(by='month_day').agg(simps)

In [311]:
cdd_groupby.head()

,cdd
month_day,
1-1,28.215600
1-10,13.672233
1-11,27.938967
1-12,43.859967
1-13,33.725133


In [312]:
cdd_groupby.loc['8-25']

cdd    61.850233
Name: 8-25, dtype: float64

In [313]:
data_df[data_df.month_day == '8-25'].cdd.sum()

61.716899999999995

In [223]:
data_df.head(24)

,month,day,hour,temp,kW,cdd,month-day
0,8,25,1,73.9,121.0925,2.3292,8-25
1,8,25,2,72.0,121.0625,2.2500,8-25
2,8,25,3,72.0,120.1400,2.2500,8-25
3,8,25,4,72.0,119.4850,2.2500,8-25
4,8,25,5,72.0,119.8200,2.2500,8-25
5,8,25,6,71.1,120.7300,2.2125,8-25
6,8,25,7,71.1,120.8900,2.2125,8-25
7,8,25,8,73.0,123.5100,2.2917,8-25
8,8,25,9,75.9,125.5825,2.4125,8-25
9,8,25,10,80.1,121.2600,2.5875,8-25


In [234]:
data_df['cdd'] = data_df.temp.apply(lambda x: max(round((x-CDD_BASE_TEMP)/24,4),0))

In [235]:
data_df['month_day'] = data_df.month.astype('str')+'-'+data_df.day.astype('str')

In [244]:
data_df[data_df.month_day == '8-25'].cdd.sum()

61.716899999999995

In [237]:
cdd_groupby.loc['8-25']

cdd    59.25855
Name: 8-25, dtype: float64

In [236]:
daily_data_df = data_df[['month_day', 'month', 'day']].drop_duplicates().reset_index(drop=True)

kW_groupby = data_df[['month_day', 'kW']].groupby(by='month_day').sum()
temp_groupby = data_df[['month_day', 'temp']].groupby(by='month_day').mean()

cdd_groupby = data_df[['month_day', 'cdd']].groupby(by='month_day').agg(simps)
# Shit, I actually need to include the first hour of the next day to properly span the needed time period for integration

daily_data_df = daily_data_df.merge(kW_groupby, on='month_day')
daily_data_df = daily_data_df.merge(temp_groupby, on='month_day')
daily_data_df = daily_data_df.merge(cdd_groupby, on='month_day')

daily_data_df.head()

,month_day,month,day,kW,temp,cdd
0,8-25,8,25,3134.3325,79.716667,59.258550
1,8-26,8,26,3345.5300,86.435417,65.741158
2,8-27,8,27,3150.2325,84.637500,64.013925
3,8-28,8,28,3063.2075,81.027083,60.545358
4,8-29,8,29,3192.7775,82.662500,62.099708


In [289]:
daily_data_df.drop(columns='cdd', inplace=True)

In [315]:
# HERE
daily_data_df = daily_data_df.merge(cdd_groupby, on='month_day', how='left')

In [318]:
daily_data_df.head()

,month_day,month,day,kW,temp,cdd
0,8-25,8,25,3134.3325,79.716667,61.850233
1,8-26,8,26,3345.5300,86.435417,68.508533
2,8-27,8,27,3150.2325,84.637500,66.623967
3,8-28,8,28,3063.2075,81.027083,63.005700
4,8-29,8,29,3192.7775,82.662500,64.772633


In [319]:
reg_data_df = daily_data_df[['month_day', 'kW', 'temp', 'cdd']].copy()

In [320]:
reg_data_df.head()

,month_day,kW,temp,cdd
0,8-25,3134.3325,79.716667,61.850233
1,8-26,3345.5300,86.435417,68.508533
2,8-27,3150.2325,84.637500,66.623967
3,8-28,3063.2075,81.027083,63.005700
4,8-29,3192.7775,82.662500,64.772633


In [321]:
X_cdd = reg_data_df['cdd'].values
y_cdd = reg_data_df['kW'].values
X_cdd = sm.add_constant(X_cdd)
mod_cdd = sm.OLS(y_cdd, X_cdd, hasconst=True)
res_cdd = mod_cdd.fit()
res_cdd.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.381
Model:                            OLS   Adj. R-squared:                  0.379
Method:                 Least Squares   F-statistic:                     215.7
Date:                Sat, 12 Oct 2019   Prob (F-statistic):           2.24e-38
Time:                        13:53:10   Log-Likelihood:                -2591.1
No. Observations:                 352   AIC:                             5186.
Df Residuals:                     350   BIC:                             5194.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1378.6876     58.523     23.558      0.000    1263.587    1493.788
x1            18.6511      1.270     14.686      0.000      16.153      21.149
==============================================================================
Omnibus:                       20.653   Durbin-Watson:                   0.314
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               23.109
Skew:                          -0.628   Prob(JB):                     9.59e-06
Kurtosis:                       3.022   Cond. No.                         133.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [322]:
X_temp = reg_data_df['temp'].values
y_temp = reg_data_df['kW'].values
X_temo = sm.add_constant(X_temp)
mod_temp = sm.OLS(y_temp, X_temp, hasconst=True)
res_temp = mod_temp.fit()
res_temp.summary()

C:\Users\natha\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1657: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ess/self.df_model


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                       inf
Date:                Sat, 12 Oct 2019   Prob (F-statistic):                nan
Time:                        13:53:10   Log-Likelihood:                -2660.2
No. Observations:                 352   AIC:                             5322.
Df Residuals:                     351   BIC:                             5326.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            34.5953      0.391     88.503      0.000      33.827      35.364
==============================================================================
Omnibus:                        3.862   Durbin-Watson:                   0.308
Prob(Omnibus):                  0.145   Jarque-Bera (JB):                3.355
Skew:                           0.155   Prob(JB):                        0.187
Kurtosis:                       2.636   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [179]:
train_df.head()

,year,month,all,residential,commercial,industrial,cdd_rel_temp,avg_temp
0,2001,8,1064,168,857,39,240.208333,25.748118
1,2002,8,1117,264,828,25,280.058333,27.001210
2,2003,8,1076,216,839,21,236.033333,25.613978
3,2004,8,1137,218,895,24,190.666667,24.110349
4,2005,8,1202,221,960,21,264.983333,26.546505


In [180]:
# train_df[['cdd_rel_temp', 'avg_temp']] = StandardScaler().fit_transform(train_df[['cdd_rel_temp', 'avg_temp']])
# # train_df['avg_temp'] = StandardScaler().fit_transform(train_df['avg_temp'])
# test_df[['cdd_rel_temp', 'avg_temp']] = StandardScaler().fit_transform(test_df[['cdd_rel_temp', 'avg_temp']])
# # test_df['avg_temp'] = StandardScaler().fit_transform(test_df['avg_temp'])

In [181]:
train_df.head()

,year,month,all,residential,commercial,industrial,cdd_rel_temp,avg_temp
0,2001,8,1064,168,857,39,240.208333,25.748118
1,2002,8,1117,264,828,25,280.058333,27.001210
2,2003,8,1076,216,839,21,236.033333,25.613978
3,2004,8,1137,218,895,24,190.666667,24.110349
4,2005,8,1202,221,960,21,264.983333,26.546505


In [182]:
X_train_cdd = train_df['cdd_rel_temp'].values
y_train_cdd = train_df['residential'].values
X_train_cdd = sm.add_constant(X_train_cdd)
mod_train_cdd = sm.OLS(y_train_cdd, X_train_cdd, hasconst=True)
res_train_cdd = mod_train_cdd.fit()
res_train_cdd.summary()

C:\Users\natha\Anaconda3\lib\site-packages\scipy\stats\stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.366
Model:                            OLS   Adj. R-squared:                  0.324
Method:                 Least Squares   F-statistic:                     8.674
Date:                Fri, 11 Oct 2019   Prob (F-statistic):             0.0100
Time:                        16:04:23   Log-Likelihood:                -76.289
No. Observations:                  17   AIC:                             156.6
Df Residuals:                      15   BIC:                             158.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         80.7803     50.989      1.584      0.134     -27.899     189.460
x1             0.6033      0.205      2.945      0.010       0.167       1.040
==============================================================================
Omnibus:                        4.799   Durbin-Watson:                   1.559
Prob(Omnibus):                  0.091   Jarque-Bera (JB):                2.449
Skew:                          -0.851   Prob(JB):                        0.294
Kurtosis:                       3.748   Cond. No.                     2.28e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.28e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [183]:
X_avg_temp = train_df['avg_temp'].values
y_avg_temp = train_df['residential'].values
X_avg_temp = sm.add_constant(X_avg_temp)
mod_avg_temp = sm.OLS(y_avg_temp, X_avg_temp, hasconst=True)
res_avg_temp = mod_avg_temp.fit()
res_avg_temp.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.361
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     8.477
Date:                Fri, 11 Oct 2019   Prob (F-statistic):             0.0107
Time:                        16:04:23   Log-Likelihood:                -76.360
No. Observations:                  17   AIC:                             156.7
Df Residuals:                      15   BIC:                             158.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -249.8401    164.920     -1.515      0.151    -601.359     101.679
x1            18.4754      6.346      2.912      0.011       4.950      32.001
==============================================================================
Omnibus:                        4.743   Durbin-Watson:                   1.574
Prob(Omnibus):                  0.093   Jarque-Bera (JB):                2.422
Skew:                          -0.851   Prob(JB):                        0.298
Kurtosis:                       3.723   Cond. No.                         770.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [184]:
X_test = test_df['cdd_rel_temp'].values
X_test = sm.add_constant(X_test)
y_pred_cdd = res_train_cdd.predict(X_test)

In [185]:
X_test = test_df['avg_temp'].values
X_test = sm.add_constant(X_test)
y_pred_avg_temp = res_avg_temp.predict(X_test)

In [186]:
y_test = test_df['residential'].values

In [187]:
mean_squared_error(y_test, y_pred_cdd)

775.541303049965

In [188]:
mean_squared_error(y_test, y_pred_avg_temp)

776.910901862546